In [1]:
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV,RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor
import warnings

from sklearn.ensemble import BaggingClassifier,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier,CatBoostRegressor
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier,StackingRegressor

In [2]:
cancer = pd.read_csv('BreastCancer.csv',index_col = 0)

In [3]:
cancer.info()

<class 'pandas.core.frame.DataFrame'>
Index: 699 entries, 61634 to 13454352
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Clump          699 non-null    int64 
 1   UniCell_Size   699 non-null    int64 
 2   Uni_CellShape  699 non-null    int64 
 3   MargAdh        699 non-null    int64 
 4   SEpith         699 non-null    int64 
 5   BareN          699 non-null    int64 
 6   BChromatin     699 non-null    int64 
 7   NoemN          699 non-null    int64 
 8   Mitoses        699 non-null    int64 
 9   Class          699 non-null    object
dtypes: int64(9), object(1)
memory usage: 60.1+ KB


In [99]:
cancer.head()

,Clump,UniCell_Size,Uni_CellShape,MargAdh,SEpith,BareN,BChromatin,NoemN,Mitoses,Class
Code,,,,,,,,,,
61634,5,4,3,1,2,2,2,3,1,Benign
63375,9,1,2,6,4,10,7,7,2,Malignant
76389,10,4,7,2,2,8,6,1,1,Malignant
95719,6,10,10,10,8,10,7,10,7,Malignant
128059,1,1,1,1,2,5,5,1,1,Benign


In [4]:
X = cancer.drop('Class',axis=1)
y = cancer['Class']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify = y)

In [5]:
knn = KNeighborsClassifier()
nb = GaussianNB()
dtc = DecisionTreeClassifier(random_state=24)
svm = SVC(random_state = 24, probability=True)
lr = LogisticRegression(random_state= 24)
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr)


In [6]:
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9714285714285714


In [7]:
y_pred_prob = stack.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_pred_prob)

0.990841384863124

In [109]:
knn = KNeighborsClassifier()
nb = GaussianNB()
dtc = DecisionTreeClassifier(random_state=24)
svm = SVC(random_state = 24, probability=True)
lr = LogisticRegression(random_state= 24)
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr)


In [111]:
params = {
    'final_estimator__max_depth': [3,4,5],
    'KNN__n_neighbors': [5,6,7],
    'SVM__C': np.linspace(0.001,3,5),
    'TREE__max_depth': [None,2,4],
    'final_estimator__n_estimators': [10,50,100],
    'passthrough': [True,False],
    
}
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [113]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=rfc)

In [115]:
gcv = GridSearchCV(stack,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 810 candidates, totalling 4050 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('KNN',
                                                       KNeighborsClassifier()),
                                                      ('NB', GaussianNB()),
                                                      ('TREE',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('SVM',
                                                       SVC(probability=True,
                                                           random_state=24))],
                                          final_estimator=RandomForestClassifier(random_state=24)),
             n_jobs=-1,
             param_grid={'KNN__n_neighbors': [5, 6, 7],
                         'SVM__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00]),
                         'TREE__max_depth': [None, 2, 4],
                         'final_estimator__max_depth': [3, 4, 5],
                         'final_estimator__n_estimators': [10, 50, 100],
                         'passthrough': [True, False]},
             scoring='neg_log_loss', verbose=3)

In [116]:
model_b = gcv.best_estimator_

In [117]:
model_b

StackingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=7)),
                               ('NB', GaussianNB()),
                               ('TREE',
                                DecisionTreeClassifier(max_depth=4,
                                                       random_state=24)),
                               ('SVM',
                                SVC(C=3.0, probability=True, random_state=24))],
                   final_estimator=RandomForestClassifier(max_depth=5,
                                                          n_estimators=50,
                                                          random_state=24),
                   passthrough=True)

In [118]:
import pickle
pkfile = open('stack_breast_cancer.pkl','wb')
pickle.dump(model_b,pkfile)

In [119]:
with open('stack_breast_cancer1.pkl', 'wb') as pkfile:
    pickle.dump(model_b, pkfile)

## Using passthrough = true option

In [9]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr,passthrough = True)
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.9714285714285714


In [10]:
y_pred_prob = stack.predict_proba(X_test)[:,1]
roc_auc_score(y_test,y_pred_prob)

0.9959742351046699

## Glass Dataset

In [12]:
glass = pd.read_csv('Glass.csv')

In [13]:
glass.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    object 
dtypes: float64(9), object(1)
memory usage: 16.8+ KB


In [14]:
X = glass.drop('Type',axis=1)
y = glass['Type']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify = y)

In [16]:
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7230769230769231


In [17]:
y_pred_prob = stack.predict_proba(X_test)
log_loss(y_test,y_pred_prob)

0.726108562583823

In [15]:
knn = KNeighborsClassifier()
nb = GaussianNB()
dtc = DecisionTreeClassifier(random_state=24)
svm = SVC(random_state = 24, probability=True)
lr = LogisticRegression(random_state= 24)
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr)


In [73]:
params = {
    'final_estimator__max_depth': [3,4,5],
    'KNN__n_neighbors': [5,6,7],
    'SVM__C': np.linspace(0.001,3,5),
    'TREE__max_depth': [None,2,4],
    'final_estimator__n_estimators': [10,50,100],
    'passthrough': [True,False],
    
}
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [75]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=rfc)

In [77]:
gcv = GridSearchCV(stack,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 810 candidates, totalling 4050 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('KNN',
                                                       KNeighborsClassifier()),
                                                      ('NB', GaussianNB()),
                                                      ('TREE',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('SVM',
                                                       SVC(probability=True,
                                                           random_state=24))],
                                          final_estimator=RandomForestClassifier(random_state=24)),
             n_jobs=-1,
             param_grid={'KNN__n_neighbors': [5, 6, 7],
                         'SVM__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00]),
                         'TREE__max_depth': [None, 2, 4],
                         'final_estimator__max_depth': [3, 4, 5],
                         'final_estimator__n_estimators': [10, 50, 100],
                         'passthrough': [True, False]},
             scoring='neg_log_loss', verbose=3)

In [81]:
print(gcv.best_score_)
gcv.best_params_

-0.7344968650237726


{'KNN__n_neighbors': 7,
 'SVM__C': 3.0,
 'TREE__max_depth': 4,
 'final_estimator__max_depth': 5,
 'final_estimator__n_estimators': 50,
 'passthrough': True}

In [83]:
model = gcv.best_estimator_

In [85]:
import pickle
pkfile = open('stack_breast_cancer.pkl','wb')
pickle.dump(model,pkfile)

## With passthrough

In [19]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=lr,passthrough = True)
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7076923076923077


C:\Users\dai.STUDENTSDC\AppData\Local\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
y_pred_prob = stack.predict_proba(X_test)
log_loss(y_test,y_pred_prob)

0.774780087213877

## Random Forest 

In [22]:
rfc = RandomForestClassifier(random_state=24)

In [23]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=rfc,passthrough = True)
stack.fit(X_train,y_train)
y_pred = stack.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7076923076923077


In [24]:
y_pred_prob = stack.predict_proba(X_test)
log_loss(y_test,y_pred_prob)

1.2222517693776012

## Grid Search

In [26]:
stack.get_params()

{'cv': None,
 'estimators': [('KNN', KNeighborsClassifier()),
  ('NB', GaussianNB()),
  ('TREE', DecisionTreeClassifier(random_state=24)),
  ('SVM', SVC(probability=True, random_state=24))],
 'final_estimator__bootstrap': True,
 'final_estimator__ccp_alpha': 0.0,
 'final_estimator__class_weight': None,
 'final_estimator__criterion': 'gini',
 'final_estimator__max_depth': None,
 'final_estimator__max_features': 'sqrt',
 'final_estimator__max_leaf_nodes': None,
 'final_estimator__max_samples': None,
 'final_estimator__min_impurity_decrease': 0.0,
 'final_estimator__min_samples_leaf': 1,
 'final_estimator__min_samples_split': 2,
 'final_estimator__min_weight_fraction_leaf': 0.0,
 'final_estimator__monotonic_cst': None,
 'final_estimator__n_estimators': 100,
 'final_estimator__n_jobs': None,
 'final_estimator__oob_score': False,
 'final_estimator__random_state': 24,
 'final_estimator__verbose': 0,
 'final_estimator__warm_start': False,
 'final_estimator': RandomForestClassifier(random_stat

In [27]:
params = {
    'final_estimator__max_depth': [3,4,5],
    'KNN__n_neighbors': [5,6,7],
    'SVM__C': np.linspace(0.001,3,5),
    'TREE__max_depth': [None,2,4],
    'final_estimator__n_estimators': [10,50,100],
    'passthrough': [True,False],
    
}
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

In [28]:
stack = StackingClassifier(estimators=[('KNN',knn),('NB',nb),('TREE',dtc),('SVM',svm)],
                           final_estimator=rfc)

In [29]:
gcv = GridSearchCV(stack,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3,n_jobs=-1)
gcv.fit(X,y)

Fitting 5 folds for each of 810 candidates, totalling 4050 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=StackingClassifier(estimators=[('KNN',
                                                       KNeighborsClassifier()),
                                                      ('NB', GaussianNB()),
                                                      ('TREE',
                                                       DecisionTreeClassifier(random_state=24)),
                                                      ('SVM',
                                                       SVC(probability=True,
                                                           random_state=24))],
                                          final_estimator=RandomForestClassifier(random_state=24)),
             n_jobs=-1,
             param_grid={'KNN__n_neighbors': [5, 6, 7],
                         'SVM__C': array([1.00000e-03, 7.50750e-01, 1.50050e+00, 2.25025e+00, 3.00000e+00]),
                         'TREE__max_depth': [None, 2, 4],
                         'final_estimator__max_depth': [3, 4, 5],
                         'final_estimator__n_estimators': [10, 50, 100],
                         'passthrough': [True, False]},
             scoring='neg_log_loss', verbose=3)

In [30]:
gcv.best_score_

-0.7344968650237726

In [31]:
gcv.best_params_

{'KNN__n_neighbors': 7,
 'SVM__C': 3.0,
 'TREE__max_depth': 4,
 'final_estimator__max_depth': 5,
 'final_estimator__n_estimators': 50,
 'passthrough': True}

In [32]:
# from tqdm import tqdm ## For progress bar
# gcv = GridSearchCV(stack,param_grid=params,cv=kfold,scoring='neg_log_loss',verbose=3,n_jobs=-1)



# for i in tqdm (range(180)):
#     gcv.fit(X,y)

In [33]:
model = gcv.best_estimator_

In [34]:
model

StackingClassifier(estimators=[('KNN', KNeighborsClassifier(n_neighbors=7)),
                               ('NB', GaussianNB()),
                               ('TREE',
                                DecisionTreeClassifier(max_depth=4,
                                                       random_state=24)),
                               ('SVM',
                                SVC(C=3.0, probability=True, random_state=24))],
                   final_estimator=RandomForestClassifier(max_depth=5,
                                                          n_estimators=50,
                                                          random_state=24),
                   passthrough=True)

## Serializing the fitted object

In [36]:
best_stack = gcv.best_estimator_

In [64]:
import pickle
pkfile = open('stack_glass.pkl','wb')
pickle.dump(best_stack,pkfile)

UNlabelled data